In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. O primeiro passo da exploração de dados é obter informações gerais, deve-se importar o arquivo “covid_19_clean_complete.csv” e obter as seguintes informações.

In [2]:
covid19 = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
covid19.head()

In [3]:
# a. Quantidade de linhas e colunas
len(covid19)   # linhas
len(covid19.columns)   # colunas

In [4]:
# b. Quais são as colunas
covid19.columns.tolist()

In [5]:
# c. O tipo de dado de cada coluna
covid19.dtypes

In [6]:
# d. A coluna de datas deve ser transformada de “Object” para “datetime64”, utilizando o comando “pd.todatetime( )”
covid19.Date = covid19.Date.astype('datetime64')

In [7]:
# e. Informações estatísticas sobre o banco de dados
covid19.describe()

In [8]:
# f. Se necessário, transformar uma coluna pertinente para formato de data

# g. Quais colunas apresentam NaN
covid19.columns[covid19.isnull().any()].tolist()

# 2. Crie um dataframe que contenha apenas as 5 províncias da China que mais registraram casos.

In [9]:
covid19 = covid19.rename(columns={'Country/Region': 'Country', 'Province/State': 'Province'})
covid19_china = covid19.loc[covid19.Country == 'China']
covid19_china.head()
#covid19_china.sort_values(by='Confirmed', ascending=False)


In [10]:
#a. Descubra qual o nome das províncias chinesas contidas no dataset
covid19_china.Province.unique().tolist()

In [11]:
#b. Retire do banco de dados apenas as informações dessas províncias
covid19_china.groupby('Province').Confirmed.agg([len, sum]).sort_values(by='sum', ascending=False)

In [12]:
#c. Tome apenas as informações das features: “Confirmed”, “Active”, “Deaths”, “Recovered”
covid19_china = covid19_china.loc[:, ['Province', 'Confirmed', 'Active', 'Deaths', 'Recovered']]
covid19_china

In [13]:
#d. Agrupe o novo dataframe a partir dessas províncias. Qual a função de agrupamento que deve ser usada?
covid19_china = covid19_china.groupby('Province')


In [14]:
#e. Produza um dataframe contendo as 5 regiões com o maior número de casos confirmados
covid19_china.sum().sort_values(by='Confirmed', ascending=False).head(5)

# 3. A coluna “Province/State” contém muitos valores faltando. Para descartá-la perdendo o mínimo de informação, inclua na coluna “Country/Region” o nome das regiões junto ao nome dos países em que existem mais de uma província registrada no banco de dados, utilizando o método “.apply”. Como por exemplo, juntar o nome China com a região pertinente: “China_Hubei”, “China_Guangdong”.


In [15]:
#a. Elabore uma função que tenha uma linha do banco de dados como argumento e, se a coluna 
#“Province/State” não for valor faltante, concatenar seu valor com coluna “Country/Region”
    # i. Utilize o método “pandas.notna” para averiguar de “Province/State” é valor faltante

def concat_columns(row):
    if pd.notna(row.Province):
        row.Country = row.Country + '_' + row.Province
    return row



In [16]:
# b. Faça uma cópia do banco de dados para poder retirar informações sem perder o banco de dados original
# c. Aplique, por meio do método “apply”, a função criada no item a
covid19_cp = covid19.apply(concat_columns, axis='columns')

In [17]:
# d. Exclua a coluna “Province/State” do novo dataframe
covid19_cp = covid19_cp.drop('Province', 1)

In [18]:
covid19_cp[pd.notna(covid19.Province)]

# 4. Importe o arquivo "world meter data.csv" e, a partir de seus dados de população e continentes, faça um ranking de maior número de mortes por milhão de habitantes entre os continentes

In [19]:
#a. Importar o banco de dados "world meter data.csv"
worldometer_data = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')
worldometer_data = worldometer_data.rename(columns={'Country/Region': 'Country'})
worldometer_data.head()

In [20]:
#b. Tome apenas as informações de população, país e continentes deste banco de dados
worldometer_data = worldometer_data.loc[:, ['Country', 'Continent', 'Population',]]
worldometer_data = worldometer_data.set_index('Country')
worldometer_data.head()

In [21]:
#c. Agrupe o “covid_19_clean_complete” por países. Deve-se usar o dataframe do item “c)” ou o original?
covid19 = covid19.groupby('Country').sum()


In [22]:
#d. Mescle os dois dataframes, a partir dos países, contendo apenas as informações de mortes, população e continente
merged_df = worldometer_data.join(covid19)
merged_df = merged_df.drop(columns=['Lat', 'Long','Confirmed', 'Recovered', 'Active'])

In [23]:
merged_df.head()

In [24]:
#e. Agrupe o novo dataset agora pelo continente
merged_df = merged_df.groupby('Continent').sum()

In [25]:
#f. Crie uma nova coluna contendo o número de mortes por milhão dentro dos continentes (Utilize a fórmula: (mortes/população)*10^6)
merged_df['Deaths_Normalized']=(merged_df.Deaths/merged_df.Population)*1E6

In [26]:
#g. Faça um ranking do número de mortes por milhão de habitantes entre os continentes
merged_df.sort_values(by = 'Deaths_Normalized', ascending=False)

# Visualização de dados

In [76]:
covid19 = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv', parse_dates=True)
covid19.head()

# 1. Criar gráfico linechart que indique a progressão no tempo das mortes nas regiões: Américas, Eastern Mediterran e Europe (OBS: Utilize apenas o arquivo “covid_19_clean_complete.csv”)


# Line Chart

In [6]:
#a. Importar bibliotecas matplotlib e seaborn, para retirar a necessidade de criar arquivos para 
#   os gráficos, digitar comando “%matplotlib inline”
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [77]:
#b. Realizar agrupamento utilizando as colunas “Date” e “WHO Region”, utilizando “sum” como função de agregação
covid19_cp = covid19.groupby(['Date', 'WHO Region']).sum()
covid19_cp.head(10)

In [78]:
#c. Plotar gráficos que contenham as progressões das regiões exigidas
plt.figure(figsize=(35,6))
ax = sns.lineplot(x="Date", y="Deaths", hue="WHO Region", data=covid19_cp)

#d. adicionar nomes para eixos, título do gráfico, grid e legendas para as linhas
plt.title("Progressao de Mortes por Covid 19")
ax.tick_params(axis='x', rotation=90)

In [ ]:
#e. Analisar gráfico de acordo com as perguntas feitas

1. Quais das regiões tiveram mais mortes?

R: As Americas e Europa

2. Qual o momento em que cada progressão começa a subir?

R: 19/03/2020 na Europa, 01/04/2020 nas Americas

3. Como está a taxa de crescimento de cada progressão?

R: A Europa começa antes e mais rapido, mas satura, enquanto as Americas crescem de forma monotonica



# Bar Charts

In [79]:
covid19_cp = covid19.groupby(['WHO Region']).sum()
covid19_cp.head()

In [88]:
plt.figure(figsize=(12,6))
sns.barplot(x=covid19_cp.index, y=covid19_cp['Deaths'])
plt.title("Mortes por Covid 19")

1. Quais os 2 continentes com maior número de mortes?

America e Europa

2. Qual o continente com menor número de mortes?

Africa

3. Qual sua hipótese para responder às perguntas anteriores?

Maior População

In [89]:
plt.figure(figsize=(12,6))
sns.barplot(x=merged_df.index, y=merged_df['Deaths_Normalized'])
plt.title("Mortes/milhão de habitantes por Covid 19")

# Scatter Plots

In [91]:
sns.regplot(x=merged_df['Deaths_Normalized'], y=merged_df['Population'])

# 1. Descobrir se há relação entre o número de mortes por milhão e a quantidade de habitantes, nos 3 continentes com maior média de mortes por milhão de habitantes, por meio de scatter plot (OBS: Utilizar somente arquivo “worldometer_data.csv”)


In [4]:
worldometer_data = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')
worldometer_data.head()

In [101]:
#a. Agrupar o dataframe pela coluna “Continent”, retirar apenas os dados de "Deaths/1 M pop", 
#   utilizar “mean” como função de agregação e ordenar para descobrir os 3 continentes com maior valor
continents = worldometer_data.loc[:, ['Continent', 'Deaths/1M pop',]].groupby('Continent').mean().sort_values(by='Deaths/1M pop', ascending=False)
continents.head()

In [125]:
#b. Obter apenas os dados do dataframe, correspondentes a esses continentes
worldometer_data = worldometer_data.loc[worldometer_data.Continent.isin(continents.index[0:3])]
worldometer_data.head()


In [126]:
#c. Criar Scatterplot sendo o eixo x mostrando as mortes por milhão, o eixo y, a população total e 
#   representar os 3 continentes com cores diferentes
sns.scatterplot(x=worldometer_data['Deaths/1M pop'], y=worldometer_data['Population'], hue=worldometer_data['Continent'])

#d. Responder as perguntas com base no gráfico

1. Onde se concentram a maioria dos dados?

Perto da origem

2. No segundo continentes por mais mortes por milhão, é possível perceber alguma relação entre as features

Na America do Sul parece haver uma correlação

3. Pelo gráfico, é possível enxergar relação entre essas colunas do banco de dados?

Nas Americas sim, Na Europa nem tanto

# Heatmap de Correlação

# 1. Será utilizado o método DataFrame.corr() do pandas para gerar os valores de correlação, então o aluno deverá produzir um heatmap para a visualização dos valores gerados


In [9]:
# a. Criar uma variável para armazenar os valores de correlação
worldometer_data = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')
worldometer_data.head()
var = worldometer_data.corr()

# b. Criar um heatmap, com título, utilizando a variável criada no item anterior
plt.figure(figsize=(14,7))
sns.heatmap(data=var, annot=True)

In [ ]:
# c. Escolher duas features para analisar seus valores de correlação (Sugestão:“Population”, “Teste/1M pop”)
#  -0.075